In [31]:
import ee

# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()


In [32]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [33]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            100.67799289874142,
            6.339057689337892
          ],
          [
            100.67596196772081,
            6.3382887311459655
          ],
          [
            100.67522051671278,
            6.336943051553064
          ],
          [
            100.67264155668596,
            6.326561976772766
          ],
          [
            100.6766711817279,
            6.325216266618256
          ],
          [
            100.67841197974587,
            6.326369732679765
          ],
          [
            100.68079751777259,
            6.326241569910792
          ],
          [
            100.68086199177264,
            6.327010546046168
          ],
          [
            100.68150673177979,
            6.327491155548643
          ],
          [
            100.68131330977798,
            6.330374803178657
          ],
          [
            100.68095870277352,
            6.331047651975837
          ],
          [
            100.68150673177979,
            6.332201105017177
          ],
          [
            100.68260278979051,
            6.331496217353248
          ],
          [
            100.68366661080216,
            6.331303975096517
          ],
          [
            100.684601483812,
            6.331848661305088
          ],
          [
            100.6842468768092,
            6.332681709689936
          ],
          [
            100.68518174981904,
            6.3354051277208185
          ],
          [
            100.68421463980837,
            6.338160571336402
          ],
          [
            100.6780251357423,
            6.339057689337892
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}

In [34]:
coords = geoJSON['features'][0]\
                ['geometry']\
                ['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [35]:
copernicus = ee.Image(ee.ImageCollection('COPERNICUS/S3/OLCI') \
              .filterBounds(aoi) \
              .filterDate(ee.Date('2020-12-1'), ee.Date('2023-03-17'))  \
              .first()\
              .clip(aoi))

In [36]:
copernicus.bandNames().getInfo()


['Oa01_radiance',
 'Oa02_radiance',
 'Oa03_radiance',
 'Oa04_radiance',
 'Oa05_radiance',
 'Oa06_radiance',
 'Oa07_radiance',
 'Oa08_radiance',
 'Oa09_radiance',
 'Oa10_radiance',
 'Oa11_radiance',
 'Oa12_radiance',
 'Oa13_radiance',
 'Oa14_radiance',
 'Oa15_radiance',
 'Oa16_radiance',
 'Oa17_radiance',
 'Oa18_radiance',
 'Oa19_radiance',
 'Oa20_radiance',
 'Oa21_radiance',
 'quality_flags']

In [37]:
import IPython.display as disp

url = copernicus.select('Oa18_radiance').getThumbURL({'min': -5000, 'max': 5000})
disp.Image(url=url, width=800)

In [38]:
sentinel5 = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') \
              .filterBounds(aoi) \
              .filterDate(ee.Date('2020-12-1'), ee.Date('2023-02-17'))  \
              .first()\
              .clip(aoi))

sentinel5.bandNames().getInfo()

['VV', 'VH', 'angle']

In [39]:
import IPython.display as disp

url = sentinel5.select('VH').getThumbURL({'min': -5000, 'max': 5000})
disp.Image(url=url, width=800)

In [40]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(copernicus.select('Oa18_radiance'),
                   sentinel5.select('VH'),
                   copernicus.select('Oa02_radiance').divide(sentinel5.select('VV')))


# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [0, -20, -20], 'max': [1, 1, 1]}, 'Chl absorption max., biogeochemistry, vegetation')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [41]:
type(m)

folium.folium.Map

In [42]:
import pandas as pd
import numpy as np

# Load a Sentinel-2 surface reflectance image collection and filter by location and date, and select the B8 band
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterBounds(aoi) \
              .filterDate('2020-01-01', '2020-12-31') \
              .select('B8','B11','B12')

# Get the first image in the collection and clip it to the AOI
image = collection.first().clip(aoi)

# Get the mean B8 value for all the pixels within the AOI
b8_value = image.select('B8').reduceRegion(ee.Reducer.toList(), aoi, 30).get('B8')
b11_value = image.select('B11').reduceRegion(ee.Reducer.toList(), aoi, 30).get('B11')
b12_value = image.select('B12').reduceRegion(ee.Reducer.toList(), aoi, 30).get('B12')

NDVI8 = np.array(b8_value.getInfo())/8000
NDVI11 = np.array(b11_value.getInfo())/8000
NDVI12 = np.array(b12_value.getInfo())/8000  
# # Print the mean B8 value
# print('Mean B8 value: ', b8_value.getInfo())

In [43]:
NDVI12

array([0.209   , 0.196375, 0.1775  , 0.121375, 0.10075 , 0.099625,
       0.0995  , 0.103125, 0.1095  , 0.121125, 0.126375, 0.125875,
       0.12075 , 0.076125, 0.035125, 0.0335  , 0.031125, 0.03025 ,
       0.03275 , 0.028375, 0.036   , 0.0475  , 0.082625, 0.134   ,
       0.19175 , 0.2035  , 0.222   , 0.200625, 0.146875, 0.19775 ,
       0.18025 , 0.21875 , 0.13475 , 0.11275 , 0.10775 , 0.093125,
       0.093   , 0.09825 , 0.101   , 0.097   , 0.115   , 0.12125 ,
       0.10725 , 0.0365  , 0.03425 , 0.03025 , 0.030875, 0.0295  ,
       0.025875, 0.02725 , 0.03175 , 0.0585  , 0.08025 , 0.152375,
       0.13375 , 0.1255  , 0.114625, 0.132   , 0.208375, 0.182375,
       0.183125, 0.13425 , 0.120375, 0.112625, 0.09625 , 0.09975 ,
       0.0975  , 0.10125 , 0.094625, 0.11125 , 0.111   , 0.11    ,
       0.04875 , 0.03575 , 0.030875, 0.032   , 0.028875, 0.026125,
       0.02725 , 0.02875 , 0.051625, 0.06325 , 0.0885  , 0.102   ,
       0.096   , 0.089   , 0.127875, 0.2705  , 0.251125, 0.165

In [44]:
LS9 = ee.Image(ee.ImageCollection('LANDSAT/LC09/C02/T2_L2') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-01-1'), ee.Date('2023-03-17')) 
                       .first() 
                       .clip(aoi))

In [45]:
LS9.bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [46]:
import IPython.display as disp

url = LS9.select('SR_B5').getThumbURL({'min': -1000, 'max': 5000})
disp.Image(url=url, width=800)

In [47]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(LS9.select('SR_B5'),
                   LS9.select('SR_B2'),
                   LS9.select('SR_B3').divide(LS9.select('SR_B5')))


# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [0, -20, 0], 'max': [0, 1, 1]}, 'NDVI Overlay')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [48]:
MODIS_NVDI = ee.Image(ee.ImageCollection('MODIS/061/MOD13Q1') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

In [49]:
MODIS_NVDI.bandNames().getInfo()

['NDVI',
 'EVI',
 'DetailedQA',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b07',
 'ViewZenith',
 'SolarZenith',
 'RelativeAzimuth',
 'DayOfYear',
 'SummaryQA']

In [50]:
url = MODIS_NVDI.select('NDVI').getThumbURL({'min': -1000, 'max': 5000})
disp.Image(url=url, width=800)

In [51]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(MODIS_NVDI.select('EVI'),
                   MODIS_NVDI.select('NDVI'),
                   MODIS_NVDI.select('sur_refl_b02').divide(MODIS_NVDI.select('EVI')))

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [20, 20, 0], 'max': [1, 1, 1]}, 'Enhanced Vegetation Index')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [52]:
# Load a Sentinel-2 surface reflectance image collection and filter by location and date, and select the B8 band
collection = ee.ImageCollection('MODIS/061/MOD13Q1') \
              .filterBounds(aoi) \
              .filterDate('2020-01-01', '2020-12-31') \
              .select('NDVI','EVI')

# Get the first image in the collection and clip it to the AOI
image = collection.first().clip(aoi)

# Get the mean B8 value for all the pixels within the AOI
NDVI_value = image.select('NDVI').reduceRegion(ee.Reducer.toList(), aoi, 30).get('NDVI')
EVI_value = image.select('EVI').reduceRegion(ee.Reducer.toList(), aoi, 30).get('EVI')


NDVI_value = np.array(NDVI_value.getInfo())/8000
EVI_value = np.array(EVI_value.getInfo())/8000
# # Print the mean B8 value
# print('Mean B8 value: ', b8_value.getInfo())

In [53]:
NDVI_value

array([0.87875 , 0.843   , 0.843   , ..., 1.000875, 1.000875, 1.000875])

In [54]:
EVI_value 

array([0.48825 , 0.47075 , 0.47075 , ..., 0.802625, 0.802625, 0.802625])

In [55]:
#For Unhealthy Data
import pandas as pd
import numpy as np
import plotly.express as px

array = NDVI12

colour = 0.09

status = []

for value in array:
    if value >= colour:
        status.append('Healthy')
    else:
        status.append('Unhealthy')

In [56]:
# create the DataFrame
Table = pd.DataFrame({'Area': array, 'Status': status})

# rearrange the columns
Table = Table[['Area', 'Status']]

# select only the 'Unhealthy' rows
Table_unhealthy = Table[Table['Status'] == 'Unhealthy']

# sort the table by 'Area' in ascending order
Table_unhealthy = Table_unhealthy.sort_values(by='Area', ascending=True)

# set the maximum number of rows to display
pd.options.display.max_rows = None

# show the plot
Table_unhealthy

,Area,Status
48,0.025875,Unhealthy
77,0.026125,Unhealthy
49,0.027250,Unhealthy
78,0.027250,Unhealthy
202,0.027375,Unhealthy
201,0.027875,Unhealthy
294,0.027875,Unhealthy
233,0.028000,Unhealthy
265,0.028250,Unhealthy
19,0.028375,Unhealthy


In [61]:
# create a plotly visualization using plotly express
fig = px.scatter(Table_unhealthy, x='Area', y=Table_unhealthy.index.sort_values())

# show the plot
fig.show()

In [62]:
#For Healthy Data
import pandas as pd
import numpy as np
import plotly.express as px

array = NDVI12

colour = 0.09

status = []

for value in array:
    if value >= colour:
        status.append('Healthy')
    else:
        status.append('Unhealthy')

In [63]:

# create the DataFrame
Table = pd.DataFrame({'Area': array, 'Status': status})

# rearrange the columns
Table = Table[['Area', 'Status']]

# select only the 'Healthy' rows
Table_Healthy = Table[Table['Status'] == 'Healthy']

# sort the table by 'Area' in ascending order
Table_Healthy = Table_Healthy.sort_values(by='Area', ascending=True)

# reset the index and sort by the new index in ascending order
Table_sorted = Table.reset_index(drop=True).sort_index()

# set the maximum number of rows to display
pd.options.display.max_rows = None

# show the plot
Table_Healthy

,Area,Status
448,0.090000,Healthy
465,0.090125,Healthy
243,0.090375,Healthy
221,0.091125,Healthy
279,0.092000,Healthy
220,0.092875,Healthy
419,0.092875,Healthy
36,0.093000,Healthy
35,0.093125,Healthy
365,0.093125,Healthy


In [64]:
# create a plotly visualization using plotly express
fig = px.scatter(Table_Healthy, x='Area', y=Table_Healthy.index.sort_values())

# show the plot
fig.show()